<a href="https://colab.research.google.com/github/Fermu25/Cursos/blob/main/Extracci%C3%B3n_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pdfplumber openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.7 MB/s eta 0:00:00


In [6]:
# -*- coding: utf-8 -*-
import re
import pdfplumber
import pandas as pd

PDF_PATH = "20250909-1638 Cotización.pdf"   # ajusta si tu archivo se llama distinto
OUT_XLSX = "Cotizacion_completa.xlsx"

money_rx = re.compile(r"^\$?\d{1,3}(?:,\d{3})*(?:\.\d{2})$")
int_rx   = re.compile(r"^\d+$")
sku_rx   = re.compile(r"^[A-Za-z0-9_.\-]+$")

# Líneas de encabezado/ruido que NO son parte del producto
noise_patterns = [
    r"\bCOTIZACIÓN\b", r"\bDIRIGIDO\b", r"\bRFC\b", r"https?://",
    r"\bVigencia\b", r"\bTITULAR\b", r"\bBANCO\b", r"\bCUENTA\b",
    r"\bCLAVE\b", r"\bSUBTOTAL\b", r"\bIVA\b", r"\bTOTAL\b",
    r"\bUNIT ELECTRONICS\b", r"\bCiudad de México\b", r"\bTERMINOS\b", r"\bcondiciones\b"
]
noise_rx = re.compile("|".join(noise_patterns), re.IGNORECASE)

def tokens_from_line(line_words):
    return [w["text"] for w in sorted(line_words, key=lambda w: w["x0"])]

def is_core_line(toks):
    # línea “núcleo” = empieza con partida (entero) y tiene al menos 2 importes ($x)
    return bool(toks and int_rx.match(toks[0]) and sum(1 for t in toks if money_rx.match(t)) >= 2)

def parse_core_line(toks):
    # estructura: [partida, ..., SKU, cantidad, precio, total]
    money_idx = [i for i, t in enumerate(toks) if money_rx.match(t)]
    if len(money_idx) < 2:
        return None
    total_i = money_idx[-1]
    price_i = money_idx[-2]
    if price_i - 1 < 0:
        return None
    cantidad_tok = toks[price_i - 1]
    if not int_rx.match(cantidad_tok):
        return None
    if price_i - 2 < 0:
        return None
    sku_tok = toks[price_i - 2]
    if not sku_rx.match(sku_tok):
        return None
    if not int_rx.match(toks[0]):
        return None
    partida = int(toks[0])
    prod_tokens = toks[1:price_i-2]  # entre Partida y SKU
    return {
        "partida": partida,
        "sku": sku_tok,
        "cantidad": int(cantidad_tok),
        "precio": toks[price_i],
        "total": toks[total_i],
        "prod_core": " ".join(prod_tokens).strip()
    }

def join_clean(parts):
    s = " ".join([p.strip() for p in parts if p and p.strip()])
    s = re.sub(r"\s+", " ", s).strip()
    return s

def looks_like_noise(line_text):
    return bool(noise_rx.search(line_text))

rows = []

with pdfplumber.open(PDF_PATH) as pdf:
    for page in pdf.pages:
        words = page.extract_words(x_tolerance=1, y_tolerance=1, keep_blank_chars=False, use_text_flow=True)
        # agrupar por renglón con tolerancia vertical
        words_sorted = sorted(words, key=lambda w: (round(w["top"],1), w["x0"]))
        lines, cur, cur_y = [], [], None
        for w in words_sorted:
            y = w["top"]
            if cur and abs(y - cur_y) > 2.5:
                lines.append(cur); cur = [w]; cur_y = y
            else:
                if not cur: cur_y = y
                cur.append(w)
        if cur: lines.append(cur)

        token_lines = [tokens_from_line(ln) for ln in lines]
        core_indices = [i for i, toks in enumerate(token_lines) if is_core_line(toks) and parse_core_line(toks)]

        for i in core_indices:
            parsed = parse_core_line(token_lines[i])
            if not parsed:
                continue

            # contexto PRE: líneas previas que no empiezan con número/importe y no son ruido
            pre_parts, j = [], i - 1
            while j >= 0:
                t = token_lines[j]
                starts_num = bool(t and int_rx.match(t[0]))
                has_money  = any(money_rx.match(x) for x in t)
                line_txt   = " ".join(t)
                if starts_num or has_money:
                    break
                if not looks_like_noise(line_txt):
                    pre_parts.insert(0, line_txt)
                j -= 1

            # contexto POST: igual regla
            post_parts, k = [], i + 1
            while k < len(token_lines):
                t = token_lines[k]
                starts_num = bool(t and int_rx.match(t[0]))
                has_money  = any(money_rx.match(x) for x in t)
                line_txt   = " ".join(t)
                if starts_num or has_money:
                    break
                if not looks_like_noise(line_txt):
                    post_parts.append(line_txt)
                k += 1

            producto = join_clean(pre_parts + [parsed["prod_core"]] + post_parts)

            rows.append({
                "Partida": parsed["partida"],
                "Producto": producto,
                "SKU": parsed["sku"],
                "Cantidad": parsed["cantidad"],
                "Precio Unitario": float(parsed["precio"].replace("$","").replace(",","")),
                "Total": float(parsed["total"].replace("$","").replace(",","")),
            })

df = pd.DataFrame(rows).sort_values(["Partida", "SKU"]).reset_index(drop=True)

# Exporta
df.to_excel(OUT_XLSX, index=False)

# Resumen útil en consola
print(f"✔ Exportado: {OUT_XLSX}")
print(f"Filas: {len(df)}, Partidas únicas: {df['Partida'].nunique()}")
print("Suma Total (calculada):", round(df["Total"].sum(), 2))


✔ Exportado: Cotizacion_completa.xlsx
Filas: 272, Partidas únicas: 272
Suma Total (calculada): 399368.65
